# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "af"
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "medium-depth-long-train" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted
dataset = "autshumato"
# dataset = "jw300"

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag
os.environ["dataset"] = dataset

# This will save it to a folder in our gdrive instead!
dataset_path = os.path.join("/content/drive/My Drive/masakhane", f"{source_language}-{target_language}", dataset)
gdrive_path = os.path.join(dataset_path, tag)
os.environ["dataset_path"] = dataset_path
os.environ["gdrive_path"] = gdrive_path

! echo "$gdrive_path"
! mkdir -p "$gdrive_path"

# change working directory to dataset directory (so that we download things to there)
os.chdir(dataset_path)

In [ ]:
! pwd

In [ ]:
# Install opus-tools
! pip install opustools-pkg

In [ ]:
if dataset == "jw300":
  # Download our corpus
  ! if ! (( test -f jw300.$src ) && ( test -f jw300.$tgt )); then opus_read -d JW300 -s "$src" -t "$tgt" -wm moses -w jw300."$src" jw300."$tgt" -q; else echo "Opus files are already present, skipping download."; fi

  # extract the corpus file
  ! if test -f JW300_latest_xml_$src-$tgt.xml.gz; then gunzip JW300_latest_xml_"$src"-"$tgt".xml.gz; elif test -f JW300_latest_xml_$tgt-$src.xml.gz; then gunzip JW300_latest_xml_$tgt-$src.xml.gz; else echo "ERROR: missing corpus file!"; fi

  # set the source and target file names according to dataset
  source_file = f"jw300.{source_language}"
  target_file = f"jw300.{target_language}"
else:
  # Add code to download your own dataset here
  # example:
  if dataset == "autshumato":
    # Download Autshumato data
    os.environ["base_download_path"] = f"https://github.com/LauraMartinus/ukuxhumana/raw/master/data/{source_language}_{target_language}"

    # Training
    ! if ! test -f "$src""$tgt"_parallel.train."$src"; then wget "$base_download_path"/"$src""$tgt"_parallel.train."$src"; else echo "$src""$tgt"_parallel.train."$src already present..."; fi
    ! if ! test -f "$src""$tgt"_parallel.train."$tgt"; then wget "$base_download_path"/"$src""$tgt"_parallel.train."$tgt"; else echo "$src""$tgt"_parallel.train."$tgt already present..."; fi

    # # Development
    ! if ! test -f "$src""$tgt"_parallel.dev."$src"; then wget "$base_download_path"/"$src""$tgt"_parallel.dev."$src"; else echo "$src""$tgt"_parallel.dev."$src already present..."; fi
    ! if ! test -f "$src""$tgt"_parallel.dev."$tgt"; then wget "$base_download_path"/"$src""$tgt"_parallel.dev."$tgt"; else echo "$src""$tgt"_parallel.dev."$tgt already present..."; fi

    # # Test
    ! if ! test -f "$src""$tgt"_parallel.test."$src"; then wget "$base_download_path"/"$src""$tgt"_parallel.test."$src"; else echo "$src""$tgt"_parallel.test."$src already present..."; fi
    ! if ! test -f "$src""$tgt"_parallel.test."$tgt"; then wget "$base_download_path"/"$src""$tgt"_parallel.test."$tgt"; else echo "$src""$tgt"_parallel.test."$tgt already present..."; fi

    # set the source and target file names according to dataset
    source_file = f"{source_language}{target_language}_parallel.train.{source_language}"
    target_file = f"{source_language}{target_language}_parallel.train.{target_language}"

In [ ]:
# Download the global test set.
! if ! test -f test.en-any.en; then wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en; else echo "test.en-any.en already present..."; fi
  
# And the specific test set for this language pair.
! if ! test -f test.en ; then wget -O test.en https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-"$tgt".en; else echo "test.en already present..."; fi
! if ! test -f test."$tgt"; then wget -O test."$tgt" https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-"$tgt"."$tgt"; else echo "test.$tgt already present..."; fi

In [ ]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

In [ ]:
import pandas as pd

# TMX file to dataframe
source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [ ]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [ ]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
force = False

os.environ["data_path"] = data_path = os.path.join("joeynmt", "data", source_language + target_language)
files = [f"{set_type}.bpe.{lang}" for set_type in ("train", "dev", "test") for lang in (source_language, target_language)]
all_files_exist = all([os.path.exists(os.path.join(data_path, f)) for f in files])

if not all_files_exist or force:
  ! pip install fuzzywuzzy
  ! pip install python-Levenshtein
  import time
  from fuzzywuzzy import process
  import numpy as np

  # reset the index of the training set after previous filtering
  df_pp.reset_index(drop=False, inplace=True)

  # Remove samples from the training data set if they "almost overlap" with the
  # samples in the test set.

  # Filtering function. Adjust pad to narrow down the candidate matches to
  # within a certain length of characters of the given sample.
  def fuzzfilter(sample, candidates, pad):
    candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
    if len(candidates) > 0:
      return process.extractOne(sample, candidates)[1]
    else:
      return np.nan

  # NOTE - This might run slow depending on the size of your training set. We are
  # printing some information to help you track how long it would take. 
  scores = []
  start_time = time.time()
  for idx, row in df_pp.iterrows():
    scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
    if idx % 1000 == 0:
      hours, rem = divmod(time.time() - start_time, 3600)
      minutes, seconds = divmod(rem, 60)
      print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

  # Filter out "almost overlapping samples"
  df_pp['scores'] = scores
  df_pp = df_pp[df_pp['scores'] < 95]

else:
  print("Files already exist...")

In [ ]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open(f"train.{source_language}", "w") as src_file, open(f"train.{target_language}", "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open(f"dev.{source_language}", "w") as src_file, open(f"dev.{target_language}", "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [ ]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [ ]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance
# Do subword NMT
! if ! (( test -f vocab.$src ) && ( test -f vocab.$tgt )); then subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt; else echo "BPE vocab files already present..."; fi

! if ! test -f train.bpe.$src; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src; else echo "train.bpe.$src already present..."; fi
! if ! test -f train.bpe.$tgt; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt; else echo "train.bpe.$tgt already present..."; fi

! if ! test -f dev.bpe.$src; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src; else echo "dev.bpe.$src already present..."; fi
! if ! test -f dev.bpe.$tgt; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt; else echo "dev.bpe.$tgt already present..."; fi

! if ! test -f test.bpe.$src; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src; else echo "test.bpe.$src already present..."; fi
! if ! test -f test.bpe.$tgt; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt; else echo "test.bpe.$tgt already present..."; fi

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! if ! test $(ls -l $data_path | grep train | wc -l) -gt 0; then cp train.* $data_path; else echo "train files already in data directory..."; fi
! if ! test $(ls -l $data_path | grep test | wc -l) -gt 0; then cp test.* $data_path; else echo "test files already in data directory..."; fi
! if ! test $(ls -l $data_path | grep dev | wc -l) -gt 0; then cp dev.* $data_path; else echo "dev files already in data directory..."; fi
! if ! test -f $data_path/bpe.codes.4000; then cp bpe.codes.4000 $data_path ; else echo "bpe.codes.4000 already in data directory..."; fi
! ls $data_path

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! if ! test -f joeynmt/data/$src$tgt/vocab.txt; then joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt ; else echo "vocab.txt already in data directory..."; fi

# Some output
! echo "BPE Target Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt

# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [ ]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "noam"            # Try switching to Elan scheduling
    learning_rate_decay_length: 5000 # number of steps to reduce by the decay factor for Elan method
    learning_rate_peak: 0.002  # peak for Elan scheduler (default: 1)
    learning_rate_warmup: 2000  # warmup steps for Elan scheduler
    learning_rate_factor: 1       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    patience: 8
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0002
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 2000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.25
    decoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.25
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [ ]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

In [ ]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "/content/drive/My Drive/masakhane/${src}-${tgt}-${tag}/joeynmt/models/${src}${tgt}_transformer/"

In [ ]:
# Output our validation accuracy
! cat "$gdrive_path/joeynmt/models/${src}${tgt}_transformer/validations.txt"

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/../joeynmt/models/${src}${tgt}_transformer/config.yaml"